# **Lab 10: Functions, vectors and lists**

In [ ]:
library(tidyverse)

Warning message:
“replacing previous import ‘vctrs::data_frame’ by ‘tibble::data_frame’ when loading ‘dplyr’”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Anatomy of a function
To write a function we should first think about the inputs and output. A function takes input(s), does something(s) to them, and then returns an output.

What are the input(s) and output of our normalize function?

In [ ]:
df$a <- (df$a - min(df$a, na.rm = TRUE)) / (max(df$a, na.rm = TRUE) - min(df$a, na.rm = TRUE))

In [ ]:
rescale01 <- function(x) {
#  ^ function name   ^ function argument (input vector)
    (x - min(x, na.rm = TRUE)) / (max(x, na.rm = TRUE) - min(x, na.rm = TRUE))
#   ^ function output
}
x = c(1:10, Inf)
rescale01(x)

[1]   0   0   0   0   0   0   0   0   0   0 NaN


We have turned up a bug in our function! But since the code now all lives in one place, we can fix the function once rather than having to chase down the bug every place that we copied and pasted the code.

In [ ]:
rescale01 = function(x) {
  rng = range(x, na.rm = TRUE, finite = TRUE)
  (x - rng[1]) / (rng[2] - rng[1])
}
rescale01(x)

[1] 0.0000000 0.1111111 0.2222222 0.3333333 0.4444444 0.5555556 0.6666667
 [8] 0.7777778 0.8888889 1.0000000       Inf

## Conditions

The condition part of the if statement must evaluate to either a single TRUE or FALSE. If it does not, you will get a warning:

In [ ]:
if (c(T, F)) { 1 }

Warning message in if (c(T, F)) {:
“the condition has length > 1 and only the first element will be used”

[1] 1

Similarly, a condition of NA will generate an error:

In [ ]:
if (NA) { 1 }

ERROR: Error in if (NA) {: missing value where TRUE/FALSE needed


### Logical operators

Often you will need to combine multiple logical conditions in an if statement. To do this we have the `&&` and `||` operators, which take the logical and and or, respectively, of several logical conditions:

In [ ]:
TRUE && FALSE && TRUE

[1] FALSE

In [ ]:
FALSE || TRUE || FALSE

[1] TRUE

There is a subtle but important difference betwen the single and double versions of these operators. The single `&` performs entrywise AND over logical vectors:

In [ ]:
c(T, T, F) & c(F, T, F)

[1] FALSE  TRUE FALSE

In contrast, the double ampersand `&&` returns `F` as soon as it encounters a value of `F`:

In [ ]:
c(T, T, T) && c(F, T, F)

[1] FALSE

It only returns `T` if it gets to the end of a vector without finding any `F` values:

In [ ]:
c(T, T, T) && c(T, T, T)

[1] TRUE

This is known as "short-circuiting": R can stop evaluating as soon as it hits one false value, since this will cause the & to return false:

In [ ]:
f = function() { print("f called"); F }
g = function() { print("g called"); T }
f() && g()

g() && f()

[1] "f called"


[1] FALSE

[1] "g called"
[1] "f called"


[1] FALSE

The or operator works similarly:

In [ ]:
g() || f()

f() || g()

[1] "g called"


[1] TRUE

[1] "f called"
[1] "g called"


[1] TRUE

### Testing for equality

Be careful when testing for equality in conditionals. The == operator will return a vector of logicals. If you want to make sure that any/all entries of a vector are TRUE, use the any() or all() functions:

In [ ]:
v1 = c(1, 2, 3)
v2 = c(1, 1, 2)
if (v1 == v2) { print("Wrong!") }
if (all(v1 == v2)) { print("All!") }
if (any(v1 == v2)) { print("Any!") }

Warning message in if (v1 == v2) {:
“the condition has length > 1 and only the first element will be used”

[1] "Wrong!"
[1] "Any!"


Also be wary of testing floating point numbers for equality:

In [ ]:
2 == sqrt(2) ^ 2

[1] FALSE

In [ ]:
sqrt(2) ^ 2

[1] 2

If you need to do this, use the `near()` function instead:

In [ ]:
near(2, sqrt(2) ^ 2)

[1] TRUE

### Multiple conditions

Sometimes you will want to check multiple conditions using an if statement. For example, let's define the function:
$$
sign(x)=\begin{cases}
-1, x<0\\
0, x=0\\
1, x>1
\end{cases}$$

The general form is

```
if (this) {
   do that
} else if (that) {
   do something else
} else {
   
}
```

## Function arguments

Functions can take multiple arguments. Generally they fall into one of two categories:

*   Data to be processed by the function, and
*   Options, which affect how the data gets processed.


### Rules for function arguments

Generally:

*   The data parameters should come first; and
*   The options should come second, and have sensible defaults.

Default parameter values are specified by the option=default notation:

In [ ]:
mean_ci <- function(x, conf = 0.95) {
  se <- sd(x) / sqrt(length(x))
  alpha <- 1 - conf
  mean(x) + se * qnorm(c(alpha / 2, 1 - alpha / 2))
}


When you call a function, you can omit the values of the default arguments. If overriding the default, you should specify the parameter you are overriding and then input the overridden value with an = in between:



```
mean_ci(c(1, 2, 3, 4), conf=.99) #yes
mean_ci(c(1, 2, 3, 4), .99)  # no

```

## Validation

When writing functions it's a good idea to validate the input -- that is, make sure it matches your assumptions about what is being passed to the function. Consider the following function which returns the weighted average of a vector:

In [ ]:
w_mean = function(x, w) {
    (x * w) / sum(w)
}

This function relies implicitly on the fact that the weight vector `w` is the same length as the input vector `x`. If it's not, you'll get a warning and unexpected behavior.

In [ ]:
w_mean(c(1,2,3), w=c(1, 2))

Warning message in x * w:
“longer object length is not a multiple of shorter object length”

[1] 0.3333333 1.3333333 1.0000000


It's best to make the assumption of equal length explicit by checking it:

In [ ]:
w_mean = function(x, w) {
    stopifnot(length(w) == length(x))
    (x * w) / sum(w)
}

In [ ]:
w_mean(c(1,2,3), w=c(1, 2))

ERROR: Error in w_mean(c(1, 2, 3), w = c(1, 2)): length(w) == length(x) is not TRUE


## ...

Some functions are designed to take a variable number of inputs. We saw this for example with the str_c function:

In [ ]:
stringr::str_c("a", "b")
stringr::str_c("a", "b", "c", "d")

[1] "ab"

[1] "abcd"

To construct a function that takes a variable number of arguments we use the `...` notation:

```
f = function(...) {
    <do something with variable arguments>
}

```
One thing you can do with the ... is pass it to another function:

In [ ]:
commas <- function(...) stringr::str_c(..., collapse = ", ")
commas(letters[1:10])

[1] "a, b, c, d, e, f, g, h, i, j"